# Localisation

In [1]:
import numpy as np

## Sense function

In [2]:
p=[0.2, 0.2, 0.2, 0.2, 0.2]
world=['green', 'red', 'red', 'green', 'green']
Z = 'red'
pHit = 0.6
pMiss = 0.2

def sense(p, Z):
    # booleans
    hits = [Z == x for x in world]

    # posterior probability
    q = [prob * (hit * pHit + (1 - hit) * pMiss) for hit, prob in zip(hits, p)]

    # normalised posterior
    sum_q = sum(q)
    q_norm = [val / sum_q for val in q]
    return q_norm

print(sense(p, Z))

[0.1111111111111111, 0.3333333333333332, 0.3333333333333332, 0.1111111111111111, 0.1111111111111111]


## Move function

In [3]:
p=[0, 1, 0, 0, 0]
world=['green', 'red', 'red', 'green', 'green']
measurements = ['red', 'green']
pHit = 0.6
pMiss = 0.2

def move(p, U):
    # modulo after division: the part of the vector that wraps around
    U = U % len(p)

    # move p by U and wrap around
    q = p[-U:] + p[:-U]
    return q

print(move(p, 1))

[0, 0, 1, 0, 0]


## Inexact move function

### Version with numpy
Yes, I know Numpy has a convolution function, this is for practice.

In [4]:
p=[0, 1, 0, 0, 0]
world=['green', 'red', 'red', 'green', 'green']
measurements = ['red', 'green']
pHit = 0.6
pMiss = 0.2
pExact = 0.8
pOvershoot = 0.1
pUndershoot = 0.1
H = [pUndershoot, pExact, pOvershoot]


def convolve(x, h):
    """
    Convolution of two signals x and h:
    y[n] = conv(x[n], h[n]) = Sigma x[k] * h[n - k]
    where k starts from (- length of h + 1) and continues
    till (length of h + length of x - 1). Thus, k goes
    between the points where signals x and h just overlap.
    """
    x = np.array(x)
    h = np.array(h)

    # position index k
    k_start = -len(h) + 1
    k_end = len(h) + len(x) # no - 1 because python indexing is exclusive
    k = np.arange(k_start, k_end)
    
    # zero-padding for x signal
    x_start = np.where(k == 0)[0][0]
    x_end = x_start + len(x)
    x_padded = np.zeros(len(k))
    x_padded[x_start:x_end] = x
    
    # zero-padding for h signal
    h_padded = np.zeros(len(k))
    h_padded[0:len(h)] = h[::-1]

    # convolve: sum of pointwise multiplication
    y = []
    h = list(h_padded)
    for n in range(0, max(k)):
        # modulo: wrap around the h vector by u
        u = n % max(k)
        # shift h vector by n and wrap around
        h_shifted = h[-u:] + h[:-u]

        # sum of the impulse responses
        y.append(sum(x_padded * h_shifted))
    
    return y


In [5]:
def move(p, U, H):
    """
    Move location vector p by U steps with
    movement probability distribution H.
    """
    if U < 0:
        movement = range(-1, (U - 1), -1)
    else:
        movement = range(U)

    for move in movement:
        # move p by U and wrap around
        p_shifted = p[-move:] + p[:-move]

        # convolve prior p_shifted and the movement probabilities H
        y = convolve(p_shifted, H)

        # slice q at the size of p
        q = y[0:len(p)]
        # surplus values from convolution
        surplus = y[len(p):]

        # wrap around
        for ind in range(len(surplus)):
            q[ind] += surplus[ind]

    return q

In [6]:
print(move(p, 1, H))

[0.0, 0.1, 0.8, 0.1, 0.0]


### Version without numpy

In [7]:
p=[0, 1, 0, 0, 0]
world=['green', 'red', 'red', 'green', 'green']
measurements = ['red', 'green']
pHit = 0.6
pMiss = 0.2
pExact = 0.8
pOvershoot = 0.1
pUndershoot = 0.1
H = [pUndershoot, pExact, pOvershoot]


def convolve(x, h):
    """
    Convolution of two signals x and h:
    y[n] = conv(x[n], h[n]) = Sigma x[k] * h[n - k]
    where k starts from (- length of h + 1) and continues
    till (length of h + length of x - 1). Thus, k goes
    between the points where signals x and h just overlap.
    """
    # position index k
    k_start = -len(h) + 1
    k_end = len(h) + len(x) # no - 1 because python indexing is exclusive
    k = range(k_start, k_end)
    
    # zero-padding for x signal
    x_start = [ind for ind, k_val in enumerate(k) if k_val == 0][0]
    x_end = x_start + len(x)
    x_padded = [0] * len(k)
    x_padded[x_start:x_end] = x
    
    # zero-padding for h signal
    h_padded = [0] * len(k)
    h_padded[0:len(h)] = h[::-1]

    # convolve: sum of pointwise multiplication
    y = []
    for n in range(0, max(k)):
        # modulo: wrap around the h vector by u
        u = n % max(k)
        # shift h vector by n and wrap around
        h_shifted = h_padded[-u:] + h_padded[:-u]

        # sum of the impulse responses
        impulse_response = [x_val * h_val for x_val, h_val in zip(x_padded, h_shifted)]
        y.append(sum(impulse_response))
    
    return y


In [8]:
print(move(p, 1, H))

[0.0, 0.1, 0.8, 0.1, 0.0]


In [9]:
# Move twice
p=[0, 1, 0, 0, 0]
p = move(p, 1, H)
print(move(p, 1, H))

[0.010000000000000002, 0.010000000000000002, 0.16000000000000003, 0.6600000000000001, 0.16000000000000003]


In [10]:
# Move 1000 times
p=[0, 1, 0, 0, 0]
for ind in range(1000):
    p = move(p, 1, H)
print(p)

[0.20000000000000365, 0.20000000000000373, 0.20000000000000365, 0.2000000000000035, 0.2000000000000035]


## Sense and move

In [11]:
p=[0.2, 0.2, 0.2, 0.2, 0.2]
world=['green', 'red', 'red', 'green', 'green']
measurements = ['red', 'green']
motions = [1,1]

def sense_move(p, measurements, motions, H):
    """
    Sense and move loop for a robot.
    :param p: probability distribution for robot location.
    :param measurements: (array) measurements from the robot's sensors.
    :param motions: (array) robot motions.
    :param H: (array) system response to motion uncertainty (probability distribution).
    """
    for ind in range(max([len(measurements), len(motions)])):
        # sense if there is a measurement
        try:
            p = sense(p, measurements[ind])
        except IndexError:
            pass
        
        # move if there is a motion
        try:
            p = move(p, motions[ind], H)
        except IndexError:
            pass

    return p
    
p = sense_move(p, measurements, motions, H)
print(p)

[0.21157894736842106, 0.1515789473684211, 0.08105263157894739, 0.16842105263157897, 0.38736842105263164]


## Bayes' rule

X = location
Z = measurement

P(X_i | Z) = P(Z | X_i) P(X_i) / P(Z)

Non-normalised posterior:  
p~(X_i | Z) = P(Z | X_i) P(X_i)  
Normaliser alpha:  
a = Sigma p~(X_i | Z)  
Normalised posterior:
P(X_i | Z) = 1/a p~(X_i | Z)

## Cancer test

In [12]:
p_cancer = 0.001    # prior of cancer
p_not_cancer = 0.999    # prior of not cancer
p_pos_cancer = 0.8  # p hit
p_pos_not_cancer = 0.1     # p false alarm

likelihood = np.array([p_pos_cancer, p_pos_not_cancer])
prior = np.array([p_cancer, p_not_cancer])

posterior_non_norm = likelihood * prior

normaliser = sum(posterior_non_norm)

posterior = 1 / normaliser * posterior_non_norm

print(posterior)

[0.00794439 0.99205561]


## Two coins quiz

In [13]:
# take coin with 50% chance of fair coin
# flip it
# observation: heads
# Q: What is the P(fair | heads) ?

p_heads_fair = 0.5  # fair coin
p_heads_loaded = 0.1    # loaded coin
p_fair = 0.5
p_loaded = 0.5

likelihood = np.array([p_heads_fair, p_heads_loaded])
prior = np.array([p_fair, p_loaded])

posterior_non_norm = likelihood * prior

normaliser = sum(posterior_non_norm)

posterior = 1 / normaliser * posterior_non_norm
p_fair_heads = posterior[0]

print(p_fair_heads)

0.8333333333333334
